# GNN + LSTM with learnable combination of correlation windows

This notebook implements a tutorial pipeline that:

- Generates synthetic price & volume time series for 3 assets (A, B, C).
- **Precomputes rolling Pearson correlations** of log-returns for a set of candidate windows.
- Feeds only **price and volume as raw node inputs** to the model.
- Supplies per-time-step per-edge candidate-window correlations to the model.
- The model contains learnable `edge_alphas` (one per edge $\\times$ number\\_of\\_candidate\\_windows).\n
  These alphas are softmaxed and used to compute a weighted combination of candidate-window correlations —
  effectively allowing the model to **fine-tune which time-window(s) are the most relevant for each edge**.

The notebook trains a GNN + LSTM pipeline to predict the next 5-minute direction of asset A (up/down).

In [97]:
# Standard imports
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from pathlib import Path

print('torch version:', torch.__version__)

torch version: 2.7.0


In [98]:

def load_asset(asset):
    freq = '5min'
    DATA_DIR = Path('../dataset')
    path = DATA_DIR / f"{asset}_{freq}.csv"

    df = pd.read_csv(path)
    df['timestamp'] = pd.to_datetime(df["system_time"])
    df['timestamp'] = df['timestamp'].dt.round('min')
    df = df.sort_values('timestamp').set_index('timestamp')
    return df[['midpoint','bids_notional_0']]  
    
df_ADA = load_asset("ADA").rename(columns={'midpoint': 'ADA', 'bids_notional_0': 'vol_ADA'})
df_BTC = load_asset("BTC").rename(columns={'midpoint': 'BTC', 'bids_notional_0': 'vol_BTC'})
df_ETH = load_asset("ETH").rename(columns={'midpoint': 'ETH', 'bids_notional_0': 'vol_ETH'})
df = df_ADA.join(df_BTC).join(df_ETH)
print(df.shape)
df.head()


(3367, 6)


,ADA,vol_ADA,BTC,vol_BTC,ETH,vol_ETH
timestamp,,,,,,
2021-04-07 11:38:00+00:00,1.17255,2461.409912,55899.995,18997.929688,1970.965,1032.089966
2021-04-07 11:43:00+00:00,1.18390,5481.839844,56111.655,8416.750000,1975.115,3847.350098
2021-04-07 11:48:00+00:00,1.17830,383.899994,56100.015,2.700000,1969.295,7542.339844
2021-04-07 11:53:00+00:00,1.18150,2243.899902,56163.445,1703.689941,1971.535,2136.570068
2021-04-07 11:58:00+00:00,1.19330,2266.129883,56229.995,1438.400024,1974.995,75441.078125


In [99]:
# columns are: ADA	vol_ADA	BTC	vol_BTC	ETH	vol_ETH

df.rename(columns={'ADA': 'price_A', 'vol_ADA': 'vol_A', 'BTC': 'price_B', 'vol_BTC': 'vol_B', 'ETH': 'price_C', 'vol_ETH': 'vol_C'}, inplace=True)
df.reset_index(inplace=True)
df.head()

,timestamp,price_A,vol_A,price_B,vol_B,price_C,vol_C
0,2021-04-07 11:38:00+00:00,1.17255,2461.409912,55899.995,18997.929688,1970.965,1032.089966
1,2021-04-07 11:43:00+00:00,1.18390,5481.839844,56111.655,8416.750000,1975.115,3847.350098
2,2021-04-07 11:48:00+00:00,1.17830,383.899994,56100.015,2.700000,1969.295,7542.339844
3,2021-04-07 11:53:00+00:00,1.18150,2243.899902,56163.445,1703.689941,1971.535,2136.570068
4,2021-04-07 11:58:00+00:00,1.19330,2266.129883,56229.995,1438.400024,1974.995,75441.078125


In [100]:
print(df.isna().sum())
print(np.isinf(df[['price_A','price_B','price_C','vol_A','vol_B','vol_C']]).sum())

timestamp    0
price_A      0
vol_A        0
price_B      2
vol_B        2
price_C      3
vol_C        3
dtype: int64
price_A    0
price_B    0
price_C    0
vol_A      0
vol_B      0
vol_C      0
dtype: int64


## 1) Generate synthetic data 📊

We produce simple random walks for prices and noisy volumes. We compute log-returns (for correlation calculation) but the model's node inputs will be only price and volume.

In [101]:
#np.random.seed(42)
#T = 3000  # number of timesteps (5-minute intervals)

# generate random-walk-like prices
#price_A = np.cumsum(np.random.normal(0, 0.5, T)) + 100.0
#price_B = np.cumsum(np.random.normal(0, 0.4, T)) + 80.0
#price_C = np.cumsum(np.random.normal(0, 0.3, T)) + 120.0

# volumes noisy
#vol_A = np.abs(np.random.normal(50, 15, T))
#vol_B = np.abs(np.random.normal(60, 18, T))
#vol_C = np.abs(np.random.normal(55, 20, T))

#df = pd.DataFrame({
#    'price_A': price_A,
#    'price_B': price_B,
#    'price_C': price_C,
#    'vol_A': vol_A,
#    'vol_B': vol_B,
#    'vol_C': vol_C,
#})

T = len(df)

# log returns for later correlation calculation
df['lr_A'] = np.log(df['price_A']).diff().fillna(0)
df['lr_B'] = np.log(df['price_B']).diff().fillna(0)
df['lr_C'] = np.log(df['price_C']).diff().fillna(0)

# direction label for A (target): up (1) if next price > current (0 otherwise)
df['dir_A_next'] = (df['price_A'].shift(-1) > df['price_A']).astype(int)

df.head()

,timestamp,price_A,vol_A,price_B,vol_B,price_C,vol_C,lr_A,lr_B,lr_C,dir_A_next
0,2021-04-07 11:38:00+00:00,1.17255,2461.409912,55899.995,18997.929688,1970.965,1032.089966,0.000000,0.000000,0.000000,1
1,2021-04-07 11:43:00+00:00,1.18390,5481.839844,56111.655,8416.750000,1975.115,3847.350098,0.009633,0.003779,0.002103,0
2,2021-04-07 11:48:00+00:00,1.17830,383.899994,56100.015,2.700000,1969.295,7542.339844,-0.004741,-0.000207,-0.002951,1
3,2021-04-07 11:53:00+00:00,1.18150,2243.899902,56163.445,1703.689941,1971.535,2136.570068,0.002712,0.001130,0.001137,1
4,2021-04-07 11:58:00+00:00,1.19330,2266.129883,56229.995,1438.400024,1974.995,75441.078125,0.009938,0.001184,0.001753,0


## 2) Precompute correlations for candidate windows 🔗

We choose a list of candidate windows (in timesteps). For each edge (A-B, A-C, C-B) we compute rolling **Pearson correlation** of the log-returns for each candidate window. The results are stored in an array of shape $\text{[T, num\_edges, num\_windows]}$.

In [102]:
# Candidate windows (in number of 5-min steps)
candidate_windows = [6, 12, 24, 48]  # e.g., 30min, 1h, 2h, 4h
n_w = len(candidate_windows)

# Undirected Edges order for corr_array: AB (0), AC (1), CB (2)
edges = [('A','B'), ('A','C'), ('C','B')]
n_edges = len(edges)

# Prepare array: corr_array[t, edge_idx, win_idx]
corr_array = np.zeros((T, n_edges, n_w), dtype=np.float32)

for wi, w in enumerate(candidate_windows):
    # Rolling correlation using pandas (center=False for causal calculation)
    # We use min_periods=1 to avoid NaNs early on. 
    r_ab = df['lr_A'].rolling(window=w, min_periods=1).corr(df['lr_B']).to_numpy()
    r_ac = df['lr_A'].rolling(window=w, min_periods=1).corr(df['lr_C']).to_numpy()
    r_cb = df['lr_C'].rolling(window=w, min_periods=1).corr(df['lr_B']).to_numpy()

    corr_array[:, 0, wi] = np.nan_to_num(r_ab, nan=0.0, posinf=0.0, neginf=0.0)
    corr_array[:, 1, wi] = np.nan_to_num(r_ac, nan=0.0, posinf=0.0, neginf=0.0)
    corr_array[:, 2, wi] = np.nan_to_num(r_cb, nan=0.0, posinf=0.0, neginf=0.0)

print('corr_array shape:', corr_array.shape)  # (T, n_edges, n_windows)



corr_array shape: (3367, 3, 4)


## 3) Dataset: sliding windows 🖼️

The dataset creates sliding windows of length `window` for both node features (`x_seq`) and edge correlation features (`edge_corr_seq`), with the target being the direction of Asset A at the first time step *after* the window.

In [103]:
class GraphWindowDatasetCorr(Dataset):
    def __init__(self, df, corr_array, window=12, normalize=True):
        super().__init__()
        self.df = df.reset_index(drop=True)
        self.corr = corr_array
        self.window = window
        self.N = len(df)
        self.normalize = normalize

        # Compute normalization stats correctly
        if normalize:
            self.price_mean = self.df[['price_A','price_B','price_C']].mean().values
            self.price_std  = self.df[['price_A','price_B','price_C']].std().values + 1e-6

            # IMPORTANT: volumes stats computed on log1p(vol)
            lv = np.log1p(self.df[['vol_A','vol_B','vol_C']].values)
            self.vol_mean = lv.mean(axis=0)
            self.vol_std  = lv.std(axis=0) + 1e-6

    def __len__(self):
        return self.N - self.window - 1

    def __getitem__(self, idx):
        x_seq = []
        edge_seq = []

        for t in range(idx, idx + self.window):
            row = self.df.iloc[t]

            price = np.array([row.price_A, row.price_B, row.price_C], dtype=np.float32)
            vol_raw = np.array([row.vol_A, row.vol_B, row.vol_C], dtype=np.float32)
            vol = np.log1p(vol_raw)

            # normalize
            price = (price - self.price_mean) / self.price_std
            vol   = (vol   - self.vol_mean)   / self.vol_std

            node_feats = np.stack([price, vol], axis=1).astype(np.float32)
            x_seq.append(torch.tensor(node_feats, dtype=torch.float32))

            edge_corrs_t = torch.tensor(self.corr[t], dtype=torch.float32)
            edge_seq.append(edge_corrs_t)

        target = int(self.df.iloc[idx + self.window].dir_A_next)
        return x_seq, edge_seq, target

# Create dataset instances with chronological splits
train_cut = int(0.7 * T)
val_cut = int(0.85 * T)

train_df = df.iloc[:train_cut].copy()
val_df = df.iloc[train_cut:val_cut].copy()
test_df = df.iloc[val_cut:].copy()

train_corr = corr_array[:train_cut]
val_corr = corr_array[train_cut:val_cut]
test_corr = corr_array[val_cut:]

window = 12
train_ds = GraphWindowDatasetCorr(train_df, train_corr, window=window)
val_ds   = GraphWindowDatasetCorr(val_df, val_corr, window=window)
test_ds  = GraphWindowDatasetCorr(test_df, test_corr, window=window)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=64)
test_loader = DataLoader(test_ds, batch_size=64)

print('Train samples:', len(train_ds), '| Val samples:', len(val_ds), '| Test samples:', len(test_ds))

Train samples: 2343 | Val samples: 492 | Test samples: 493


## 4) Model: Edge-weighted GNN + LSTM 🧠

This model incorporates the novel **learnable window combination** into the GNN's message passing:

- The GNN message from $u \to v$ uses concatenation $[h_u, e_{uv}]$, where $e_{uv}$ is a scalar edge feature.
- $e_{uv}$ is computed as a **softmax-weighted sum** of the candidate-window correlations available at time $t$, with the weights determined by the learned `edge_alphas` ($\alpha_{i, w}$). 
- LSTM processes the sequence of node A embeddings for final prediction.

In [104]:
class EdgeWeightedGNNLayer(nn.Module):
    def __init__(self, in_dim, out_dim, n_edges):
        super().__init__()
        # message MLP that takes [h_u, e_uv_scalar] (in_dim + 1)
        self.msg_mlp = nn.Sequential(
            nn.Linear(in_dim + 1, out_dim),
            nn.ReLU(),
        )
        # update MLP that takes [h_v, aggregated_message] (in_dim + out_dim)
        self.update_mlp = nn.Sequential(
            nn.Linear(in_dim + out_dim, out_dim),
            nn.ReLU(),
        )
        self.n_edges = n_edges
    
    def forward(self, x, edge_index, edge_scalars):
        # x: [num_nodes, in_dim]
        # edge_index: tensor [2, num_directed_edges]
        # edge_scalars: length num_directed_edges vector (the combined correlation)
        num_nodes = x.size(0)
        device = x.device
        
        # initialize aggregated messages
        agg = torch.zeros((num_nodes, self.msg_mlp[0].out_features), device=device)
        
        row, col = edge_index
        num_e = row.size(0)
        
        # Naive loop over directed edges to compute messages and aggregation
        for ei in range(num_e):
            u = row[ei].item()
            v = col[ei].item()
            e_scalar = edge_scalars[ei].view(1)  # scalar tensor
            
            # Message construction: [h_u, e_uv]
            mu = torch.cat([x[u], e_scalar], dim=0)  # [in_dim + 1]
            msg = self.msg_mlp(mu)
            agg[v] += msg # Sum aggregation (can be mean, max, etc.)
            
        # Update step: [h_v, aggregated_message]
        new_x = self.update_mlp(torch.cat([x, agg], dim=1))
        return new_x

class GNN_LSTM_EdgeFineTune(nn.Module):
    def __init__(self, in_dim, gnn_dim, lstm_dim, n_edges, n_windows, edge_index):
        super().__init__()
        self.gnn1 = EdgeWeightedGNNLayer(in_dim, gnn_dim, n_edges)
        self.gnn2 = EdgeWeightedGNNLayer(gnn_dim, gnn_dim, n_edges)
        self.lstm = nn.LSTMCell(gnn_dim, lstm_dim)
        self.classifier = nn.Linear(lstm_dim, 2)
        
        # Learnable alpha per UNDIRECTED edge to combine windows
        # Shape: [n_undirected_edges, n_windows]
        self.edge_alphas = nn.Parameter(torch.randn(n_edges, n_windows) * 0.1)
        self.edge_index = edge_index.clone()  
        self.n_edges = n_edges
        self.n_windows = n_windows
    
    def combine_edge_windows(self, edge_corrs_t):
        """
        Combines candidate correlations using learned softmax weights.
        edge_corrs_t: tensor [n_undirected_edges, n_windows]
        returns directed_scalars: tensor [num_directed_edges] matching edge_index order
        """
        # Softmax over windows for each undirected edge
        weights = F.softmax(self.edge_alphas, dim=1)  # [n_edges, n_windows]
        # Weighted sum: gives one scalar per undirected edge
        e_scalar_undirected = torch.sum(weights * edge_corrs_t, dim=1)  # [n_edges]
        
        # Map undirected scalar to directed edges (assuming: AB, AC, CB order)
        # Mapping list matches the predefined edge_index order:
        # (0->1, 0->2, 1->0, 1->2, 2->0, 2->1) -> (AB, AC, BA, BC, CA, CB)
        # AB/BA uses index 0 (AB), AC/CA uses index 1 (AC), BC/CB uses index 2 (CB)
        directed_map = [0, 1, 0, 2, 1, 2] 
        directed_scalars = e_scalar_undirected[directed_map]
        
        return directed_scalars # shape [num_directed_edges]
    
    def forward(self, x_seq, edge_corr_seq):
        # x_seq: list length K of [3, in_dim]
        # edge_corr_seq: list length K of [n_undirected_edges, n_windows]

         # quick check shapes & nan/inf
        if any([torch.isnan(x_t).any() or torch.isinf(x_t).any() for x_t in x_seq]):
            print("FOUND NaN/Inf in x_seq")
        if any([torch.isnan(e).any() or torch.isinf(e).any() for e in edge_corr_seq]):
            print("FOUND NaN/Inf in edge_corr_seq")
        
        h = None
        c = None
        
        # Transfer edge_index to device (if not already there)
        edge_index_d = self.edge_index.to(self.edge_alphas.device)
        
        for x_t, edge_corrs_t in zip(x_seq, edge_corr_seq):
            # Get the combined correlation scalar for each directed edge at time t
            edge_scalars = self.combine_edge_windows(edge_corrs_t.to(self.edge_alphas.device))
            
            # Run GNN layers
            x_t_d = x_t.to(self.edge_alphas.device)
            z = self.gnn1(x_t_d, edge_index_d, edge_scalars)
            z = self.gnn2(z, edge_index_d, edge_scalars)
            
            # Take node A embedding (index 0) and run LSTM
            node_A = z[0]
            if h is None:
                h = torch.zeros_like(node_A)
                c = torch.zeros_like(node_A)
            h, c = self.lstm(node_A, (h, c))
            
        # Final classification based on the last hidden state h
        logits = self.classifier(h)
        return logits

## 5) Model Setup ⚙️

Define the graph structure and initialize the model, optimizer, and loss function.

In [105]:
# Directed edge index matching the combine_edge_windows assumption:
# Order: 0->1(AB), 0->2(AC), 1->0(BA), 1->2(BC), 2->0(CA), 2->1(CB)
edge_index = torch.tensor([
    [0, 0, 1, 1, 2, 2],
    [1, 2, 0, 2, 0, 1]
], dtype=torch.long)

n_edges = 3         # Undirected edges (AB, AC, CB) 
n_windows = len(candidate_windows)
in_dim = 2          # [price, vol]
gnn_dim = 32
lstm_dim = 32

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GNN_LSTM_EdgeFineTune(in_dim, gnn_dim, lstm_dim, n_edges, n_windows, edge_index).to(device)
optimizer = optim.Adam(model.parameters(), lr=3e-4)
criterion = nn.CrossEntropyLoss()
print('Model and optimizer ready. Device:', device)

Model and optimizer ready. Device: cpu


In [106]:
# run one sample forward for debug
x_seq, edge_seq, target = train_ds[0]
# cast to device + dtype
x_seq = [x.float().to(device) for x in x_seq]
edge_seq = [e.float().to(device) for e in edge_seq]
model.eval()
with torch.no_grad():
    logits = model(x_seq, edge_seq)
    print("logits:", logits)
    print("any nan in logits:", torch.isnan(logits).any())
    # print edge alpha weights
    print("edge alphas (raw):", model.edge_alphas)
    print("edge alphas softmax:", F.softmax(model.edge_alphas, dim=1))


FOUND NaN/Inf in x_seq
logits: tensor([nan, nan])
any nan in logits: tensor(True)
edge alphas (raw): Parameter containing:
tensor([[-0.0660, -0.0598,  0.0901,  0.1373],
        [-0.0489, -0.1252, -0.0879,  0.1046],
        [-0.1324,  0.0406,  0.1200,  0.0702]], requires_grad=True)
edge alphas softmax: tensor([[0.2272, 0.2287, 0.2656, 0.2785],
        [0.2467, 0.2285, 0.2372, 0.2876],
        [0.2127, 0.2529, 0.2738, 0.2605]])


In [107]:
# --- PATCH: ensure dtype, normalization, edge_alphas init, lr and grad clip ---
# 1) dataset already changed to float32 and log1p on volumes (see earlier instructions)

# 2) re-init model with zeros for alphas and small lr
model = GNN_LSTM_EdgeFineTune(in_dim, gnn_dim, lstm_dim, n_edges, n_windows, edge_index).to(device)
with torch.no_grad():
    model.edge_alphas.zero_()   # uniform start

optimizer = optim.Adam(model.parameters(), lr=3e-5)  # smaller LR
criterion = nn.CrossEntropyLoss()

# 3) training loop change: per-sample to-device cast + nan check + grad clip
for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    n_samples = 0
    for x_seq_batch, edge_seq_batch, target_batch in train_loader:
        optimizer.zero_grad()
        batch_logits = []
        batch_targets = []
        for i in range(len(target_batch)):
            # cast to device & float
            x_s = [xi[i].float().to(device) for xi in x_seq_batch]
            e_s = [ei[i].float().to(device) for ei in edge_seq_batch]

            logits = model(x_s, e_s)

            if torch.isnan(logits).any() or torch.isinf(logits).any():
                print("NaN/Inf logits detected BEFORE loss. Aborting.")
                # Print diagnostics
                print("edge_alphas:", model.edge_alphas)
                raise RuntimeError("NaN logits")

            batch_logits.append(logits.unsqueeze(0))
            batch_targets.append(target_batch[i].unsqueeze(0))

        batch_logits = torch.cat(batch_logits, dim=0)
        batch_targets = torch.cat(batch_targets, dim=0).to(device)

        loss = criterion(batch_logits, batch_targets)
        if torch.isnan(loss) or torch.isinf(loss):
            print("Loss is NaN/Inf. Stopping")
            raise RuntimeError("Loss NaN")

        loss.backward()

        # clip grads
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        # check grads for nan
        for name, p in model.named_parameters():
            if p.grad is not None and (torch.isnan(p.grad).any() or torch.isinf(p.grad).any()):
                print("NaN/Inf in grad of", name)
                raise RuntimeError("NaN grad detected!")

        optimizer.step()
        running_loss += loss.item() * len(target_batch)
        n_samples += len(target_batch)
    print("Epoch done. avg loss:", running_loss / n_samples)


FOUND NaN/Inf in x_seq
NaN/Inf logits detected BEFORE loss. Aborting.
edge_alphas: Parameter containing:
tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]], requires_grad=True)


RuntimeError: NaN logits

## 6) Training loop (fine-tuning) 🚀

We train both the GNN/LSTM weights and the **learnable $\alpha$ parameters** simultaneously. The softmaxed weights are printed to observe which windows the model prioritizes.

In [108]:
def evaluate_accuracy(loader):
    model.eval()
    total = 0
    correct = 0
    with torch.no_grad():
        # NOTE: The DataLoader produces batches of lists (not tensors) for x_seq and edge_seq.
        # We must manually iterate over the batch dimension.
        for x_seq_batch, edge_seq_batch, target_batch in loader:
            for i in range(len(target_batch)):
                # Extract sequence for i-th sample in batch
                x_s = [xi[i] for xi in x_seq_batch] # list of [3, in_dim]
                e_s = [ei[i] for ei in edge_seq_batch] # list of [n_edges, n_windows]
                
                logits = model(x_s, e_s)
                if torch.isnan(logits).any() or torch.isinf(logits).any():
                    print("NaN logits detected")
                    # print some debugging info
                    for j, xj in enumerate(x_s[:3]):
                        print("x_s[{}] stats: min {:.3e}, max {:.3e}, nan {}".format(j, xj.min().item(), xj.max().item(), torch.isnan(xj).any()))
                    for j, ej in enumerate(e_s[:3]):
                        print("e_s[{}] stats: min {:.3e}, max {:.3e}, nan {}".format(j, ej.min().item(), ej.max().item(), torch.isnan(ej).any()))
                    raise RuntimeError("Stopping due to NaN logits")
                pred = logits.argmax().item()
                correct += int(pred == int(target_batch[i].item()))
                total += 1
    return correct / total if total>0 else 0

EPOCHS = 10
for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    n_samples = 0
    
    for x_seq_batch, edge_seq_batch, target_batch in train_loader:
        optimizer.zero_grad()
        batch_logits = []
        batch_targets = []
        
        # Naive per-sample processing (needed due to list-of-sequences input)
        for i in range(len(target_batch)):
            # Extract sequence for i-th sample in batch
            x_s = [xi[i] for xi in x_seq_batch]
            e_s = [ei[i] for ei in edge_seq_batch]
            
            logits = model(x_s, e_s)
            batch_logits.append(logits.unsqueeze(0))
            batch_targets.append(target_batch[i].unsqueeze(0))
            
        batch_logits = torch.cat(batch_logits, dim=0)
        batch_targets = torch.cat(batch_targets, dim=0).to(device)
        
        loss = criterion(batch_logits, batch_targets)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * len(target_batch)
        n_samples += len(target_batch)
        
    avg_loss = running_loss / n_samples
    val_acc = evaluate_accuracy(val_loader)
    
    # Show softmaxed alphas for interpretability
    with torch.no_grad():
        weights = F.softmax(model.edge_alphas, dim=1).cpu().numpy()
        
    print(f'Epoch {epoch+1}/{EPOCHS} | Loss {avg_loss:.4f} | Val Acc {val_acc:.4f}')
    for ei, e in enumerate(edges):
        print(f'  Edge {e} weights over windows {candidate_windows}:', np.round(weights[ei], 3))
    print('---')

print('\n--- Testing ---')
test_acc = evaluate_accuracy(test_loader)
print('Final Test Acc:', f"{test_acc:.4f}")

FOUND NaN/Inf in x_seq
FOUND NaN/Inf in x_seq
FOUND NaN/Inf in x_seq
FOUND NaN/Inf in x_seq
FOUND NaN/Inf in x_seq
FOUND NaN/Inf in x_seq
FOUND NaN/Inf in x_seq
FOUND NaN/Inf in x_seq
FOUND NaN/Inf in x_seq
FOUND NaN/Inf in x_seq
FOUND NaN/Inf in x_seq
FOUND NaN/Inf in x_seq
FOUND NaN/Inf in x_seq
FOUND NaN/Inf in x_seq
FOUND NaN/Inf in x_seq
FOUND NaN/Inf in x_seq
FOUND NaN/Inf in x_seq
FOUND NaN/Inf in x_seq
FOUND NaN/Inf in x_seq
FOUND NaN/Inf in x_seq
FOUND NaN/Inf in x_seq
FOUND NaN/Inf in x_seq
FOUND NaN/Inf in x_seq
FOUND NaN/Inf in x_seq
FOUND NaN/Inf in x_seq
FOUND NaN/Inf in x_seq
FOUND NaN/Inf in x_seq
FOUND NaN/Inf in x_seq
FOUND NaN/Inf in x_seq
FOUND NaN/Inf in x_seq
FOUND NaN/Inf in x_seq
FOUND NaN/Inf in x_seq
FOUND NaN/Inf in x_seq
FOUND NaN/Inf in x_seq
FOUND NaN/Inf in x_seq
FOUND NaN/Inf in x_seq
FOUND NaN/Inf in x_seq
FOUND NaN/Inf in x_seq
FOUND NaN/Inf in x_seq
FOUND NaN/Inf in x_seq
FOUND NaN/Inf in x_seq
FOUND NaN/Inf in x_seq
FOUND NaN/Inf in x_seq
FOUND NaN/I

RuntimeError: Stopping due to NaN logits

## 7) Notes and Extensions 📝

- **Real Dataset Transformation:** To use this on real data, compute `df['lr_*']` from real price series, compute candidate-window rolling correlations, and build `GraphWindowDatasetCorr` on the resulting DataFrame and correlation array.
- **Window Selection:** Choose a sensible range of candidate windows (e.g., in steps: $[6, 12, 24, 48, 96]$) relevant to your prediction horizon.
- **Alternative Parametric Approach:** Instead of discrete windows, one could parametrize an **Exponential Moving Average (EMA)** with a learnable decay parameter $\alpha$ and compute correlations on EMA-smoothed returns, though this requires differentiable rolling operations.
- **Production Considerations:** The forward pass needs **vectorization** (batching) using $\text{PyTorch Geometric}$ or custom broadcasting with `torch.scatter` for efficiency. Also, consider online fine-tuning cadence and replay buffer to avoid catastrophic forgetting.

logits: tensor([-0.0176,  0.1566], device='mps:0')
any nan in logits: tensor(False, device='mps:0')
edge alphas (raw): Parameter containing:
tensor([[ 0.0559, -0.0186, -0.1006,  0.0755],
        [-0.1034, -0.0473,  0.0094,  0.0391],
        [ 0.0753, -0.0198,  0.1452,  0.0028]], device='mps:0',
       requires_grad=True)
edge alphas softmax: tensor([[0.2629, 0.2441, 0.2248, 0.2681],
        [0.2309, 0.2443, 0.2585, 0.2663],
        [0.2556, 0.2325, 0.2742, 0.2378]], device='mps:0')
